In [ ]:
#create synthetic audio signals 

import numpy as np
import scipy.signal as signal

Fs = 44100
t = np.arange(0, Fs * 1 + 1) / Fs
w = 2

# Pulse train generation
def pulse_train(t, delay_vector, width):
    return np.sum([signal.square(2 * np.pi * (1/width) * (t - d)) for d in delay_vector], axis=0)

d1 = np.arange(w/2, Fs*2, w*210)
rect_pulse_train = pulse_train(t, d1, w)

s1 = np.sin(2 * np.pi * 300 * t)
s2 = signal.sawtooth(2 * np.pi * 200 * t)
s3 = np.sin(2 * np.pi * 500 * t)
s4 = signal.sawtooth(2 * np.pi * 150 * t, 0.5)

# Simulate FM using chirps (approximation)
x = signal.chirp(t, f0=300, f1=475, t1=1, method='linear')
x2 = signal.chirp(t, f0=200, f1=345, t1=1, method='linear')
x3 = signal.chirp(t, f0=500, f1=613, t1=1, method='linear')
x4 = signal.chirp(t, f0=150, f1=300, t1=1, method='linear')

s1 = np.sin(2 * np.pi * 300 * t)
s2 = signal.sawtooth(2 * np.pi * 200 * t)
s3 = np.sin(2 * np.pi * 500 * t)
s4 = signal.sawtooth(2 * np.pi * 150 * t, 0.5)

# Simulate FM using chirps (approximation)
x = signal.chirp(t, f0=300, f1=475, t1=1, method='linear')
x2 = signal.chirp(t, f0=200, f1=345, t1=1, method='linear')
x3 = signal.chirp(t, f0=500, f1=613, t1=1, method='linear')
x4 = signal.chirp(t, f0=150, f1=300, t1=1, method='linear')



In [ ]:
# convert time to freq and normalize

import librosa

FFTwindow = 2000
hop = FFTwindow

def compute_stft(sig):
    return np.abs(librosa.stft(sig, n_fft=FFTwindow, hop_length=hop, window='hann')).T

combo1 = np.tile(np.concatenate([x[:2000], x[:2000]]), 20)
combo2 = np.tile(np.concatenate([x3[:4000], x4[:4000]]), 10)

COMBO1 = compute_stft(combo1)
COMBO2 = compute_stft(combo2)

data = np.concatenate([COMBO1, COMBO2], axis=0)
mean = np.mean(data, axis=0)
std = np.std(data, axis=0)
norm_data = (data - mean) / std


In [ ]:
# define autoencoder

import tensorflow as tf
from tensorflow.keras import layers, models

visible_dim = norm_data.shape[1]
hidden_dim = 2

# train

input_layer = layers.Input(shape=(visible_dim,))
encoded = layers.Dense(hidden_dim, activation='sigmoid')(input_layer)
decoded = layers.Dense(visible_dim, activation='sigmoid')(encoded)

autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(norm_data, norm_data, epochs=10, batch_size=32)


In [ ]:
# generate data
generated = autoencoder.predict(norm_data[0:1])
generated_denorm = generated * std + mean
generated_audio = librosa.istft(generated_denorm.T, hop_length=hop, window='hann')

import soundfile as sf
sf.write("generated_audio.wav", generated_audio, Fs)
